In [1]:
#|default_exp core

# Claudette's source

This is the 'literate' source code for Claudette. You can view the fully rendered version of the notebook [here](https://claudette.answer.ai/core.html), or you can clone the git repo and run the [interactive notebook](https://github.com/AnswerDotAI/claudette/blob/main/00_core.ipynb) in Jupyter. The notebook is converted the [Python module claudette/core.py](https://github.com/AnswerDotAI/claudette/blob/main/claudette/core.py) using [nbdev](https://nbdev.fast.ai/). The goal of this source code is to both create the Python module, and also to teach the reader *how* it is created, without assuming much existing knowledge about Claude's API.

Most of the time you'll see that we write some source code *first*, and then a description or discussion of it *afterwards*.

## Setup

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [4]:
import os
# os.environ['ANTHROPIC_LOG'] = 'debug'

To print every HTTP request and response in full, uncomment the above line. This functionality is provided by Anthropic's SDK.

In [5]:
#| export
import inspect, typing, mimetypes, base64, json
from collections import abc
try: from IPython import display
except: display=None

from anthropic import Anthropic, AnthropicBedrock, AnthropicVertex
from anthropic.types import Usage, TextBlock, Message, ToolUseBlock
from anthropic.resources import messages

import toolslm
from toolslm.funccall import *

from fastcore import imghdr
from fastcore.meta import delegates
from fastcore.utils import *

In [6]:
#| hide
from nbdev import show_doc

In [7]:
#| export
empty = inspect.Parameter.empty

In [8]:
inspect.Parameter.empty

inspect._empty

:::{.callout-tip}

If you're reading the rendered version of this notebook, you'll see an "Exported source" collapsible widget below. If you're reading the source notebook directly, you'll see `#| exports` at the top of the cell. These show that this piece of code will be exported into the python module that this notebook creates. No other code will be included -- any other code in this notebook is just for demonstration, documentation, and testing.

You can toggle expanding/collapsing the source code of all exported sections by using the `</> Code` menu in the top right of the rendered notebook page.

:::

In [9]:
#| exports
models = 'claude-3-opus-20240229','claude-3-5-sonnet-20240620','claude-3-haiku-20240307'

These are the current versions of Anthropic's model at the time of writing.

In [10]:
model = models[-1]

For examples, we'll use Sonnet 3.5, since it's awesome.

## Antropic SDK

In [11]:
cli = Anthropic()

This is what Anthropic's SDK provides for interacting with Python. To use it, pass it a list of *messages*, with *content* and a *role*. The roles should alternate between *user* and *assistant*.

:::{.callout-tip}

After the code below you'll see an indented section with an orange vertical line on the left. This is used to show the *result* of running the code above. Because the code is running in a Jupyter Notebook, we don't have to use `print` to display results, we can just type the expression directly, as we do with `r` here.

:::

In [12]:
m = {'role': 'user', 'content': "I'm Jeremy"}
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Message(id='msg_01CRQ13Poufq6dUnSiSCDDAG', content=[TextBlock(text="It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=32))

### Formatting output

That output is pretty long and hard to read, so let's clean it up. We'll start by pulling out the `Content` part of the message.
To do that, we're going to write our first function which will be included to the `claudette/core.py` module.

:::{.callout-tip}

This is the first exported public function or class we're creating (the previous export was of a variable). In the rendered version of the notebook for these you'll see 4 things, in this order (unless the symbol starts with a single `_`, which indicates it's *private*):

- The signature (with the symbol name as a heading, with a horizontal rule above)
- A table of paramater docs (if provided)
- The doc string (in italics).
- The source code (in a collapsible "Exported source" block)

After that, we generally provide a bit more detail on what we've created, and why, along with a sample usage.

:::

In [13]:
r

Message(id='msg_01CRQ13Poufq6dUnSiSCDDAG', content=[TextBlock(text="It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=32))

In [14]:
type(type)

type

In [15]:
#| exports
def find_block(r:abc.Mapping, # The message to look in
               blk_type:type=TextBlock  # The type of block to find
              ):
    "Find the first block of type `blk_type` in `r.content`."
    return first(o for o in r.content if isinstance(o,blk_type))

This makes it easier to grab the needed parts of Claude's responses, which can include multiple pieces of content. By default, we look for the first text block. That will generally have the content we want to display.

In [16]:
type(r) == abc.Mapping

False

In [17]:
r

Message(id='msg_01CRQ13Poufq6dUnSiSCDDAG', content=[TextBlock(text="It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(input_tokens=10, output_tokens=32))

In [18]:
find_block(r)

TextBlock(text="It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", type='text')

In [19]:
blk = find_block(r)

In [20]:
blk

TextBlock(text="It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", type='text')

In [21]:
blk.text

"It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?"

In [22]:
#| exports
def contents(r):
    "Helper to get the contents from Claude response `r`."
    blk = find_block(r)
    if not blk and r.content: blk = r.content[0]
    return blk.text.strip() if hasattr(blk,'text') else blk

For display purposes, we often just want to show the text itself.

In [23]:
contents(r)

"It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?"

In [24]:
#| exports
@patch
def _repr_markdown_(self:(Message)):
    det = '\n- '.join(f'{k}: {v}' for k,v in self.model_dump().items())
    return f"""{contents(self)}

<details>

- {det}

</details>"""

Jupyter looks for a `_repr_markdown_` method in displayed objects; we add this in order to display just the content text, and collapse full details into a hideable section. Note that `patch` is from [fastcore](https://fastcore.fast.ai/), and is used to add (or replace) functionality in an existing class. We pass the class(es) that we want to patch as type annotations to `self`. In this case, `_repr_markdown_` is being added to Anthropic's `Message` class, so when we display the message now we just see the contents, and the details are hidden away in a collapsible details block.

In [25]:
r

It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?

<details>

- id: msg_01CRQ13Poufq6dUnSiSCDDAG
- content: [{'text': "It's nice to meet you, Jeremy! I'm Claude, an AI assistant created by Anthropic. How can I assist you today?", 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 10, 'output_tokens': 32}

</details>

One key part of the response is the `usage` key, which tells us how many tokens we used by returning a `Usage` object.

We'll add some helpers to make things a bit cleaner for creating and formatting these objects.

In [26]:
r.usage

Usage(input_tokens=10, output_tokens=32)

In [27]:
#| exports
def usage(inp=0, # Number of input tokens
          out=0  # Number of output tokens
         ):
    "Slightly more concise version of `Usage`."
    return Usage(input_tokens=inp, output_tokens=out)

The constructor provided by Anthropic is rather verbose, so we clean it up a bit, using a lowercase version of the name.

In [28]:
usage(5)

Usage(input_tokens=5, output_tokens=0)

In [29]:
#| exports
@patch(as_prop=True)
def total(self:Usage): return self.input_tokens+self.output_tokens

Adding a `total` property to `Usage` makes it easier to see how many tokens we've used up altogether.

In [30]:
usage(5,1).total

6

In [31]:
#| exports
@patch
def __repr__(self:Usage): return f'In: {self.input_tokens}; Out: {self.output_tokens}; Total: {self.total}'

In python, patching `__repr__` lets us change how an object is displayed. (More generally, methods starting and ending in `__` in Python are called `dunder` methods, and have some `magic` behavior -- such as, in this case, changing how an object is displayed.)

In [32]:
r.usage

In: 10; Out: 32; Total: 42

In [33]:
#| exports
@patch
def __add__(self:Usage, b):
    "Add together each of `input_tokens` and `output_tokens`"
    return usage(self.input_tokens+b.input_tokens, self.output_tokens+b.output_tokens)

And, patching `__add__` lets `+` work on a `Usage` object.

In [34]:
r.usage+r.usage

In: 20; Out: 64; Total: 84

### Creating messages

Creating correctly formatted `dict`s from scratch every time isn't very handy, so next up we'll add helpers for this.

In [35]:
def mk_msg(content, role='user', **kw):
    return dict(role=role, content=content, **kw)

We make things a bit more convenient by writing a function to create a message for us.

:::{.callout-note}

You may have noticed that we didn't export the `mk_msg` function (i.e. there's no "Exported source" block around it). That's because we'll need more functionality in our final version than this version has -- so we'll be defining a more complete version later. Rather than refactoring/editing in notebooks, often it's helpful to simply gradually build up complexity by re-defining a symbol.

:::

In [36]:
prompt = "I'm Jeremy"
m = mk_msg(prompt)
m

{'role': 'user', 'content': "I'm Jeremy"}

In [37]:
r = cli.messages.create(messages=[m], model=model, max_tokens=100)
r

Nice to meet you, Jeremy! I'm glad we've connected. As an AI assistant, I'm here to help you with a variety of tasks and topics. Please let me know if there's anything I can assist you with.

<details>

- id: msg_01XXyY8EzSMHvSNv7wEoYbcE
- content: [{'text': "Nice to meet you, Jeremy! I'm glad we've connected. As an AI assistant, I'm here to help you with a variety of tasks and topics. Please let me know if there's anything I can assist you with.", 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 10, 'output_tokens': 50}

</details>

In [38]:
#| exports
def mk_msgs(msgs:list, **kw):
    "Helper to set 'assistant' role on alternate messages."
    if isinstance(msgs,str): msgs=[msgs]
    return [mk_msg(o, ('user','assistant')[i%2], **kw) for i,o in enumerate(msgs)]

LLMs, including Claude, don't actually have state, but instead dialogs are created by passing back all previous prompts and responses every time. With Claude, they always alternate *user* and *assistant*. Therefore we create a function to make it easier to build up these dialog lists.

But to do so, we need to update `mk_msg` so that we can't only pass a `str` as `content`, but can also pass a `dict` or an object with a `content` attr, since these are both types of message that Claude can create. To do so, we check for a `content` key or attr, and use it if found.

In [39]:
def mk_msg(content, role='user', **kw):
    "Helper to create a `dict` appropriate for a Claude message. `kw` are added as key/value pairs to the message"
    if hasattr(content, 'content'): content,role = content.content,content.role
    if isinstance(content, abc.Mapping): content=content['content']
    return dict(role=role, content=content, **kw)

In [40]:
msgs = mk_msgs([prompt, r, 'I forgot my name. Can you remind me please?'])
msgs

[{'role': 'user', 'content': "I'm Jeremy"},
 {'role': 'assistant',
  'content': [TextBlock(text="Nice to meet you, Jeremy! I'm glad we've connected. As an AI assistant, I'm here to help you with a variety of tasks and topics. Please let me know if there's anything I can assist you with.", type='text')]},
 {'role': 'user', 'content': 'I forgot my name. Can you remind me please?'}]

Now, if we pass this list of messages to Claude, the model treats it as a conversation to respond to.

In [41]:
cli.messages.create(messages=msgs, model=model, max_tokens=200)

I'm afraid I don't actually know your name. When you introduced yourself earlier, you said your name was Jeremy. But if you've forgotten your name, I can't remind you of something I don't know. Is there anything else I can try to help you with?

<details>

- id: msg_01FHgUjp1B13fimsBC7K2jMq
- content: [{'text': "I'm afraid I don't actually know your name. When you introduced yourself earlier, you said your name was Jeremy. But if you've forgotten your name, I can't remind you of something I don't know. Is there anything else I can try to help you with?", 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 74, 'output_tokens': 59}

</details>

## Client

In [42]:
#| exports
class Client:
    def __init__(self, model, cli=None):
        "Basic Anthropic messages client."
        self.model,self.use = model,usage()
        self.c = (cli or Anthropic())

We'll create a simple `Client` for `Anthropic` which tracks usage stores the model to use. We don't add any methods right away -- instead we'll use `patch` for that so we can add and document them incrementally.

In [43]:
c = Client(model)
c.use

In: 0; Out: 0; Total: 0

In [44]:
#| exports
@patch
def _r(self:Client, r:Message, prefill=''):
    "Store the result of the message and accrue total usage."
    if prefill:
        blk = find_block(r)
        blk.text = prefill + (blk.text or '')
    self.result = r
    self.use += r.usage
    return r

We use a `_` prefix on private methods, but we document them here in the interests of literate source code.

`_r` will be used each time we get a new result, to track usage and also to keep the result available for later.

In [45]:
c._r(r)
c.use

In: 10; Out: 50; Total: 60

Whereas OpenAI's models use a `stream` parameter for streaming, Anthropic's use a separate method. We implement Anthropic's approach in a private method, and then use a `stream` parameter in `__call__` for consistency:

In [46]:
#| exports
@patch
def _stream(self:Client, msgs:list, prefill='', **kwargs):
    with self.c.messages.stream(model=self.model, messages=mk_msgs(msgs), **kwargs) as s:
        if prefill: yield(prefill)
        yield from s.text_stream
        self._r(s.get_final_message(), prefill)

Claude supports adding an extra `assistant` message at the end, which contains the *prefill* -- i.e. the text we want Claude to assume the response starts with. However Claude doesn't actually repeat that in the response, so for convenience we add it.

In [47]:
#| exports
@patch
@delegates(messages.Messages.create)
def __call__(self:Client,
             msgs:list, # List of messages in the dialog
             sp='', # The system prompt
             temp=0, # Temperature
             maxtok=4096, # Maximum tokens
             prefill='', # Optional prefill to pass to Claude as start of its response
             stream:bool=False, # Stream response?
             **kwargs):
    "Make a call to Claude."
    pref = [prefill.strip()] if prefill else []
    if not isinstance(msgs,list): msgs = [msgs]
    msgs = mk_msgs(msgs+pref)
    if stream: return self._stream(msgs, prefill=prefill, max_tokens=maxtok, system=sp, temperature=temp, **kwargs)
    res = self.c.messages.create(
        model=self.model, messages=msgs, max_tokens=maxtok, system=sp, temperature=temp, **kwargs)
    self._r(res, prefill)
    return self.result

Defining `__call__` let's us use an object like a function (i.e it's *callable*). We use it as a small wrapper over `messages.create`.

In [48]:
c = Client(model)
c.use

In: 0; Out: 0; Total: 0

In [49]:
c.model = models[-1]

In [50]:
c('Hi')

Hello! How can I assist you today?

<details>

- id: msg_01NYPFRahCLq3oMtSJcPmeDa
- content: [{'text': 'Hello! How can I assist you today?', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 8, 'output_tokens': 12}

</details>

In [51]:
c.use

In: 8; Out: 12; Total: 20

Let's try out *prefill*:

In [52]:
q = "Concisely, what is the meaning of life?"
pref = 'According to Douglas Adams,'

In [53]:
c(q, prefill=pref)

According to Douglas Adams,  "The answer to the ultimate question of life, the universe, and everything is 42."

<details>

- id: msg_01B6CTckLjEzYtciKfPziCxD
- content: [{'text': 'According to Douglas Adams,  "The answer to the ultimate question of life, the universe, and everything is 42."', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 24, 'output_tokens': 23}

</details>

We can pass `stream=True` to stream the response back incrementally:

In [54]:
for o in c('Hi', stream=True): print(o, end='')

Hello! How can I assist you today?

In [55]:
c.use

In: 40; Out: 47; Total: 87

In [56]:
for o in c(q, prefill=pref, stream=True): print(o, end='')

According to Douglas Adams,  "The answer to the ultimate question of life, the universe, and everything is 42."

In [57]:
c.use

In: 64; Out: 70; Total: 134

## Tool use

Let's now add tool use (aka *function calling*).

In [58]:
#| exports
def mk_tool_choice(choose:Union[str,bool,None])->dict:
    "Create a `tool_choice` dict that's 'auto' if `choose` is `None`, 'any' if it is True, or 'tool' otherwise"
    return {"type": "tool", "name": choose} if isinstance(choose,str) else {'type':'any'} if choose else {'type':'auto'}

In [59]:
print(mk_tool_choice('sums'))
print(mk_tool_choice(True))
print(mk_tool_choice(None))

{'type': 'tool', 'name': 'sums'}
{'type': 'any'}
{'type': 'auto'}


Claude can be forced to use a particular tool, or select from a specific list of tools, or decide for itself when to use a tool. If you want to force a tool (or force choosing from a list), include a `tool_choice` param with a dict from `mk_tool_choice`.

For testing, we need a function that Claude can call; we'll write a simple function that adds numbers together, and will tell us when it's being called:

In [60]:
def sums(
    a:int,  # First thing to sum
    b:int=1 # Second thing to sum
) -> int: # The sum of the inputs
    "Adds a + b."
    print(f"Finding the sum of {a} and {b}")
    return a + b

In [61]:
a,b = 604542,6458932
pr = f"What is {a}+{b}?"
sp = "You are a summing expert."

Claudette can autogenerate a schema thanks to the `toolslm` library. We'll force the use of the tool using the function we created earlier.

In [62]:
tools=[get_schema(sums)]
choice = mk_tool_choice('sums')

We'll start a dialog with Claude now. We'll store the messages of our dialog in `msgs`. The first message will be our prompt `pr`, and we'll pass our `tools` schema.

In [63]:
msgs = mk_msgs(pr)
r = c(msgs, sp=sp, tools=tools, tool_choice=choice)
r

ToolUseBlock(id='toolu_01M42NRewzvy1GBkXBTPJWFp', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')

<details>

- id: msg_01NYhC9rUCVdoprag3zsHuv9
- content: [{'id': 'toolu_01M42NRewzvy1GBkXBTPJWFp', 'input': {'a': 604542, 'b': 6458932}, 'name': 'sums', 'type': 'tool_use'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: tool_use
- stop_sequence: None
- type: message
- usage: {'input_tokens': 493, 'output_tokens': 53}

</details>

When Claude decides that it should use a tool, it passes back a `ToolUseBlock` with the name of the tool to call, and the params to use.

We don't want to allow it to call just any possible function (that would be a security disaster!) so we create a *namespace* -- that is, a dictionary of allowable function names to call.

In [64]:
#| exports
def _mk_ns(*funcs:list[callable]) -> dict[str,callable]:
    "Create a `dict` of name to function in `funcs`, to use as a namespace"
    return {f.__name__:f for f in funcs}

In [65]:
ns = _mk_ns(sums)
ns

{'sums': <function __main__.sums(a: int, b: int = 1) -> int>}

In [66]:
#| exports
def call_func(fc:ToolUseBlock, # Tool use block from Claude's message
              ns:Optional[abc.Mapping]=None, # Namespace to search for tools, defaults to `globals()`
              obj:Optional=None # Object to search for tools
             ):
    "Call the function in the tool response `tr`, using namespace `ns`."
    if ns is None: ns=globals()
    if not isinstance(ns, abc.Mapping): ns = _mk_ns(*ns)
    func = getattr(obj, fc.name, None)
    if not func: func = ns[fc.name]
    res = func(**fc.input)
    return dict(type="tool_result", tool_use_id=fc.id, content=str(res))    

We can now use the function requested by Claude. We look it up in `ns`, and pass in the provided parameters.

In [67]:
fc = find_block(r, ToolUseBlock)
res = call_func(fc, ns=ns)
res

Finding the sum of 604542 and 6458932


{'type': 'tool_result',
 'tool_use_id': 'toolu_01M42NRewzvy1GBkXBTPJWFp',
 'content': '7063474'}

In [68]:
#| exports
def mk_toolres(
    r:abc.Mapping, # Tool use request response from Claude
    ns:Optional[abc.Mapping]=None, # Namespace to search for tools
    obj:Optional=None # Class to search for tools
    ):
    "Create a `tool_result` message from response `r`."
    cts = getattr(r, 'content', [])
    res = [mk_msg(r)]
    tcs = [call_func(o, ns=ns, obj=obj) for o in cts if isinstance(o,ToolUseBlock)]
    if tcs: res.append(mk_msg(tcs))
    return res

In order to tell Claude the result of the tool call, we pass back the tool use assistant request and the `tool_result` response.

In [69]:
tr = mk_toolres(r, ns=ns)
tr

Finding the sum of 604542 and 6458932


[{'role': 'assistant',
  'content': [ToolUseBlock(id='toolu_01M42NRewzvy1GBkXBTPJWFp', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_01M42NRewzvy1GBkXBTPJWFp',
    'content': '7063474'}]}]

We add this to our dialog, and now Claude has all the information it needs to answer our question.

In [70]:
msgs += tr
contents(c(msgs, sp=sp, tools=tools))

'The sum of 604542 and 6458932 is 7063474.'

This works with methods as well -- in this case, use the object itself for `ns`:

In [71]:
class Dummy:
    def sums(
        self,
        a:int,  # First thing to sum
        b:int=1 # Second thing to sum
    ) -> int: # The sum of the inputs
        "Adds a + b."
        print(f"Finding the sum of {a} and {b}")
        return a + b

In [72]:
tools = [get_schema(Dummy.sums)]
o = Dummy()

msgs = mk_msgs(pr)
r = c(msgs, sp=sp, tools=tools, tool_choice=choice)
tr = mk_toolres(r, obj=o)
msgs += tr
contents(c(msgs, sp=sp, tools=tools))

Finding the sum of 604542 and 6458932


'The sum of 604542 and 6458932 is 7063474.'

## Chat

Rather than manually adding the responses to a dialog, we'll create a simple `Chat` class to do that for us, each time we make a request. We'll also store the system prompt and tools here, to avoid passing them every time.

In [73]:
#| exports
class Chat:
    def __init__(self,
                 model:Optional[str]=None, # Model to use (leave empty if passing `cli`)
                 cli:Optional[Client]=None, # Client to use (leave empty if passing `model`)
                 sp='', # Optional system prompt
                 tools:Optional[list]=None, # List of tools to make available to Claude
                 tool_choice:Optional[dict]=None): # Optionally force use of some tool
        "Anthropic chat client."
        assert model or cli
        self.c = (cli or Client(model))
        self.h,self.sp,self.tools,self.tool_choice = [],sp,tools,tool_choice

    @property
    def use(self): return self.c.use

The class stores the `Client` that will provide the responses in `c`, and a history of messages in `h`.

In [74]:
sp = "Never mention what tools you use."
chat = Chat(model, sp=sp)
chat.c.use, chat.h

(In: 0; Out: 0; Total: 0, [])

In [75]:
#| exports
@patch
def _stream(self:Chat, res):
    yield from res
    self.h += mk_toolres(self.c.result, ns=self.tools, obj=self)

In [76]:
#| exports
@patch
def __call__(self:Chat,
             pr=None,  # Prompt / message
             temp=0, # Temperature
             maxtok=4096, # Maximum tokens
             stream=False, # Stream response?
             prefill='', # Optional prefill to pass to Claude as start of its response
             **kw):
    if pr and self.h and nested_idx(self.h, -1, 'role')=='user':
        self() # There's already a user request pending, so complete it
    if pr: self.h.append(mk_msg(pr))
    if self.tools: kw['tools'] = [get_schema(o) for o in self.tools]
    if self.tool_choice: kw['tool_choice'] = mk_tool_choice(self.tool_choice)
    res = self.c(self.h, stream=stream, prefill=prefill, sp=self.sp, temp=temp, maxtok=maxtok, **kw)
    if stream: return self._stream(res)
    self.h += mk_toolres(self.c.result, ns=self.tools, obj=self)
    return res

The `__call__` method just passes the request along to the `Client`, but rather than just passing in this one prompt, it appends it to the history and passes it all along. As a result, we now have state!

In [77]:
chat("I'm Jeremy")
chat("What's my name?")

Your name is Jeremy, as you mentioned earlier.

<details>

- id: msg_01TcLNi4xhofPufafbto8NE4
- content: [{'text': 'Your name is Jeremy, as you mentioned earlier.', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 76, 'output_tokens': 13}

</details>

Let's try out prefill too:

In [78]:
q = "Concisely, what is the meaning of life?"
pref = 'According to Douglas Adams,'

In [79]:
chat(q, prefill=pref)

According to Douglas Adams,  "The answer to the ultimate question of life, the universe, and everything is 42."

<details>

- id: msg_01GvD3rhkCmn8Ydg3ZnaJiy1
- content: [{'text': 'According to Douglas Adams,  "The answer to the ultimate question of life, the universe, and everything is 42."', 'type': 'text'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: end_turn
- stop_sequence: None
- type: message
- usage: {'input_tokens': 109, 'output_tokens': 23}

</details>

In [80]:
chat = Chat(model, sp=sp)
for o in chat("I'm Jeremy", stream=True): print(o, end='')

It's nice to meet you, Jeremy! I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything I can assist you with.

In [81]:
for o in chat(q, prefill=pref, stream=True): print(o, end='')

According to Douglas Adams,  "The answer to the ultimate question of life, the universe, and everything is 42."

### Chat tool use

We automagically get streamlined tool use as well:

In [82]:
pr = f"What is {a}+{b}?"
pr

'What is 604542+6458932?'

In [83]:
chat = Chat(model, sp=sp, tools=[sums])
r = chat(pr)
r

Finding the sum of 604542 and 6458932


ToolUseBlock(id='toolu_014JuXX1MjxXFnrBYpe3sL8J', input={'a': 604542, 'b': 6458932}, name='sums', type='tool_use')

<details>

- id: msg_011rW4DbLaV5YzshGNB6byeu
- content: [{'id': 'toolu_014JuXX1MjxXFnrBYpe3sL8J', 'input': {'a': 604542, 'b': 6458932}, 'name': 'sums', 'type': 'tool_use'}]
- model: claude-3-haiku-20240307
- role: assistant
- stop_reason: tool_use
- stop_sequence: None
- type: message
- usage: {'input_tokens': 398, 'output_tokens': 72}

</details>

In [ ]:
chat()

It should be correct, because it actually used our Python function to do the addition. Let's check:

In [ ]:
a+b

## Images

Claude can handle image data as well. As everyone knows, when testing image APIs you have to use a cute puppy.

In [ ]:
# Image is Cute_dog.jpg from Wikimedia
fn = Path('samples/puppy.jpg')
display.Image(filename=fn, width=200)

In [ ]:
img = fn.read_bytes()

In [ ]:
#| exports
def img_msg(data:bytes)->dict:
    "Convert image `data` into an encoded `dict`"
    img = base64.b64encode(data).decode("utf-8")
    mtype = mimetypes.types_map['.'+imghdr.what(None, h=data)]
    r = dict(type="base64", media_type=mtype, data=img)
    return {"type": "image", "source": r}

Anthropic have documented the particular `dict` structure that expect image data to be in, so we have a little function to create that for us.

In [ ]:
#| exports
def text_msg(s:str)->dict:
    "Convert `s` to a text message"
    return {"type": "text", "text": s}

A Claude message can be a list of image and text parts. So we've also created a helper for making the text parts.

In [ ]:
q = "In brief, what color flowers are in this image?"
msg = mk_msg([img_msg(img), text_msg(q)])

In [ ]:
c([msg])

In [ ]:
#| exports
def _mk_content(src):
    "Create appropriate content data structure based on type of content"
    if isinstance(src,str): return text_msg(src)
    if isinstance(src,bytes): return img_msg(src)
    return src

There's not need to manually choose the type of message, since we figure that out from the data of the source data.

In [ ]:
_mk_content('Hi')

In [ ]:
#| exports
def mk_msg(content, # A string, list, or dict containing the contents of the message
           role='user', # Must be 'user' or 'assistant'
           **kw):
    "Helper to create a `dict` appropriate for a Claude message. `kw` are added as key/value pairs to the message"
    if hasattr(content, 'content'): content,role = content.content,content.role
    if isinstance(content, abc.Mapping): content=content['content']
    if not isinstance(content, list): content=[content]
    content = [_mk_content(o) for o in content] if content else '.'
    return dict(role=role, content=content, **kw)

When we construct a message, we now use `_mk_content` to create the appropriate parts. Since a dialog contains multiple messages, and a message can contain multiple content parts, to pass a single message with multiple parts we have to use a list containing a single list:

In [ ]:
c([[img, q]])

:::{.callout-note}

As promised (much!) earlier, we've now finally completed our definition of `mk_msg`, and this version is the one we export to the Python module.

:::

## Third party providers

### Amazon Bedrock

These are Amazon's current Claude models:

In [ ]:
#| export
models_aws = ('anthropic.claude-3-haiku-20240307-v1:0', 'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-opus-20240229-v1:0', 'anthropic.claude-3-5-sonnet-20240620-v1:0')

We don't need any extra code to support Amazon Bedrock -- we just have to set up the approach client:

In [ ]:
ab = AnthropicBedrock(
    aws_access_key=os.environ['AWS_ACCESS_KEY'],
    aws_secret_key=os.environ['AWS_SECRET_KEY'],
)
client = Client(models_aws[-1], ab)

In [ ]:
chat = Chat(cli=client)

In [ ]:
chat("I'm Jeremy")

### Google Vertex

In [ ]:
#| export
models_goog = 'claude-3-haiku@20240307', 'claude-3-sonnet@20240229', 'claude-3-opus@20240229', 'claude-3-5-sonnet@20240620'

In [ ]:
from anthropic import AnthropicVertex
import google.auth

In [ ]:
project_id = google.auth.default()[1]
region = "us-east5"
gv = AnthropicVertex(project_id=project_id, region=region)
client = Client(models_goog[-1], gv)

In [ ]:
chat = Chat(cli=client)

In [ ]:
chat("I'm Jeremy")

## Export -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()